In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
from tensorflow import keras

In [32]:
from keras.layers import Input, Lambda, Dense, Flatten

In [33]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np 
from glob import glob 
import matplotlib.pyplot as plt 

In [34]:
IMAGE_SIZE = [224, 224]

train_path = 'Dataset/chest_xray/train'
test_path = 'Dataset/chest_xray/test'

In [35]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [36]:
for layer in vgg.layers:
    layer.trainable = False


In [54]:
folders = glob('Dataset/chest_xray/train/*')
x = Flatten()(vgg.output)


In [55]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [56]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [57]:
from keras.preprocessing.image import ImageDataGenerator

In [58]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Dataset/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('Dataset/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [59]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=1,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
    verbose = 1
)


522/522 [==============================] - 774s 1s/step - loss: 0.2396 - accuracy: 0.9199 - val_loss: 0.3107 - val_accuracy: 0.9054


In [60]:
import tensorflow as tf 
from keras.models import load_model

In [61]:
model.save('pneumonia_detection.h5')

In [62]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [63]:
model= load_model('pneumonia_detection.h5')
img = image.load_img('Dataset/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
classes = model.predict(img_data)
result = int(classes[0][0])

In [65]:
if result > 0.5:
    print('Result is Normal')
else:
    print('Person is affected by Pneumonia')



Result is Normal
